Начну с того, что надо бы определиться с данными и откуда их брать.

"3.	Обучить модель на данных о числе новых выявленных случаев заболевания covid-19 в период с 01.01.2020 по 01.12.2020 в странах: Россия, Италия, Германия и Франция. Для каждой страны не учитывать дни до того как заболеваемость превысит 100 случаев в день!"


Ссылку не нашёл в задании, сам набродил на два датасета из 1 репозитория, а именно https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv и https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/jhu/full_data.csv. Мне сначала показалось, что можно брать любой, так как full_data.csv содержит общие данные о случаях заболевания, смертях и так далее, а owid-covid-data.csv включает, помимо основных показателей, дополнительные переменные. Но это не все их отличия. В full data меньше данных, но зато нет полок (owid-covid-data.csv выглядит как заборчик, если строить график заражений по дате). В full data минимальная дата - эир 2020-01-22, что не подходит под условия задачи казалось бы, но на самом деле, если убрать "до того как заболеваемость превысит 100 случаев в день", то всё ок. В итоге работал с full_data.csv

In [ ]:
countries = ["Russia", "Italy", "Germany", "France"]

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/jhu/full_data.csv"
df = pd.read_csv(url, parse_dates=["date"])
df.head()


,date,location,new_cases,new_deaths,total_cases,total_deaths,weekly_cases,weekly_deaths,biweekly_cases,biweekly_deaths
0,2020-02-24,Afghanistan,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
1,2020-02-25,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
2,2020-02-26,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
3,2020-02-27,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
4,2020-02-28,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
# просто хочу проверить, что все страны есть
uniq = set(df['location'].unique())
missing = [c for c in countries if c not in uniq]
assert missing == []

In [ ]:
print("первая дата:", df['date'].min(), "последняя дата:", df['date'].max())

первая дата: 2020-01-22 00:00:00 последняя дата: 2023-03-09 00:00:00


Теперь надо пройтись по колонкам (оставить только нужные) и убрать лишние строки, так как "Для каждой страны не учитывать дни до того как заболеваемость превысит 100 случаев в день!"

In [ ]:
df.columns

Index(['date', 'location', 'new_cases', 'new_deaths', 'total_cases',
       'total_deaths', 'weekly_cases', 'weekly_deaths', 'biweekly_cases',
       'biweekly_deaths'],
      dtype='object')

In [ ]:
need_columns = ["location", "date", "new_cases", "total_cases"]

In [ ]:
df2 = df.loc[df["location"].isin(countries), need_columns].copy()
df2.head()

,location,date,new_cases,total_cases
80938,France,2020-01-24,2.0,2.0
80939,France,2020-01-25,1.0,3.0
80940,France,2020-01-26,0.0,3.0
80941,France,2020-01-27,0.0,3.0
80942,France,2020-01-28,1.0,4.0


In [ ]:
min_date_porog = (df2[df2["new_cases"] > 100]
       .groupby("location")["date"].min()
       .rename("okey_row"))
min_date_porog

,okey_row
location,
France,2020-03-05
Germany,2020-03-05
Italy,2020-02-26
Russia,2020-03-25


Интересно почему Французы настолько позже

In [ ]:
df_res = (df2.merge(min_date_porog, on="location")
          .loc[lambda x: x["date"] >= x["okey_row"]]
          .drop(columns="okey_row")
          .sort_values(["location","date"])
          .reset_index(drop=True))
df_res.head()

,location,date,new_cases,total_cases
0,France,2020-03-05,138.0,426.0
1,France,2020-03-06,190.0,616.0
2,France,2020-03-07,332.0,948.0
3,France,2020-03-08,177.0,1125.0
4,France,2020-03-09,286.0,1411.0


In [ ]:
# перепроверочка, что данные правильно почистил
df_res.groupby("location").agg(date_min=("date","min"),
                                  date_max=("date","max"),
                                  rows=("date","size"))

,date_min,date_max,rows
location,,,
France,2020-03-05,2023-03-09,1100
Germany,2020-03-05,2023-03-09,1100
Italy,2020-02-26,2023-03-09,1108
Russia,2020-03-25,2023-03-09,1080


Из ссылки, которую Вы кидали https://www.pymc.io/projects/examples/en/latest/case_studies/bayesian_workflow.html#id1 беру версии библиотек (там примерно внизу страницы ячейка)

In [ ]:
%pip uninstall -y jax jaxlib opencv-python opencv-python-headless opencv-contrib-python cudf-polars-cu12 thinc

Found existing installation: jax 0.7.2
Uninstalling jax-0.7.2:
  Successfully uninstalled jax-0.7.2
Found existing installation: jaxlib 0.7.2
Uninstalling jaxlib-0.7.2:
  Successfully uninstalled jaxlib-0.7.2
Found existing installation: thinc 8.3.6
Uninstalling thinc-8.3.6:
  Successfully uninstalled thinc-8.3.6


In [ ]:
%pip install -qU pip setuptools wheel
%pip install -qU numpy==2.0.2 pymc==5.23.0 arviz==0.21.0 matplotlib==3.10.3 plotly==6.1.2 polars==1.30.0 watermark==2.5.0 jedi>=0.16

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.8.7 requires thinc<8.4.0,>=8.3.4, which is not installed.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
distrax 0.1.5 requires jax>=0.1.55, which is not installed.
distrax 0.1.5 requires jaxlib>=0.1.67, which is not installed.
chex 0.1.90 requires jax>=0.4.27, which is not installed.
chex 0.1.90 requires jaxlib>=0.4.27, which is not installed.


In [ ]:
%pip install -qU  watermark==2.5.0

In [ ]:
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import polars as pl
import pymc as pm

In [ ]:
from scipy import stats as sps

import pytensor
import pytensor.tensor as pt
from pytensor.scan.basic import scan
from pytensor.tensor.conv import conv2d

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w


Last updated: Wed Oct 29 2025

Python implementation: CPython
Python version       : 3.12.12
IPython version      : 7.34.0

pymc      : 5.23.0
polars    : 1.30.0
pytensor  : 2.31.7
matplotlib: 3.10.3
scipy     : 1.16.2
numpy     : 2.0.2
plotly    : 6.1.2
pandas    : 2.2.2
arviz     : 0.21.0

Watermark: 2.5.0



Начнём с того мы используем pytensor и в офиц доке на будущее нашёл путь, где лежит conv2d

In [ ]:
from scipy import stats as sps

import pytensor
import pytensor.tensor as pt
from pytensor.scan.basic import scan
from pytensor.tensor.conv import conv2d

In [ ]:
help(conv2d)

Help on function conv2d in module pytensor.tensor.conv.abstract_conv:

conv2d(input, filters, input_shape=None, filter_shape=None, border_mode='valid', subsample=(1, 1), filter_flip=True, filter_dilation=(1, 1), num_groups=1, unshared=False)
    This function will build the symbolic graph for convolving a mini-batch of a
    stack of 2D inputs with a set of 2D filters. The implementation is modelled
    after Convolutional Neural Networks (CNN).


    Parameters
    ----------
    input: symbolic 4D tensor
        Mini-batch of feature map stacks, of shape
        (batch size, input channels, input rows, input columns).
        See the optional parameter ``input_shape``.

    filters: symbolic 4D or 6D tensor
        Set of filters used in CNN layer of shape
        (output channels, input channels, filter rows, filter columns)
        for normal convolution and
        (output channels, output rows, output columns, input channels,
        filter rows, filter columns)
        for unsha

get_delay_distribution из covid/patients.py почти не меняю, мини фиксы, чтобы работало с моими версиями библиотек

In [ ]:
import os
from scipy import stats as sps
import numpy as np
import pandas as pd

def get_patient_data(file_path=None, max_delay=60):
    """ Finds every valid delay between symptom onset and report confirmation
        from the patient line list and returns all the delay samples. """
    if not file_path:
        file_path = os.path.join(os.path.dirname(__file__), "../data/patients.tar.gz")
    patients = pd.read_csv(
        file_path,
        parse_dates=False,
        usecols=["country", "date_onset_symptoms", "date_confirmation"],
        low_memory=False,
    )

    patients.columns = ["Country", "Onset", "Confirmed"]
    patients.Country = patients.Country.astype("category")

    # There's an errant reversed date
    patients = patients.replace("01.31.2020", "31.01.2020")
    patients = patients.replace("31.04.2020", "01.05.2020")

    # Only keep if both values are present
    patients = patients.dropna()

    # Must have strings that look like individual dates
    # "2020.03.09" is 10 chars long
    is_ten_char = lambda x: x.str.len().eq(10)
    patients = patients[is_ten_char(patients.Confirmed) & is_ten_char(patients.Onset)]

    # Convert both to datetimes
    patients.Confirmed = pd.to_datetime(
        patients.Confirmed, format="%d.%m.%Y", errors="coerce"
    )
    patients.Onset = pd.to_datetime(patients.Onset, format="%d.%m.%Y", errors="coerce")

    # Only keep records where confirmed > onset
    patients = patients[patients.Confirmed > patients.Onset]

    # Mexico has many cases that are all confirmed on the same day regardless
    # of onset date, so we filter it out.
    patients = patients[patients.Country.ne("Mexico")]

    # Remove any onset dates from the last two weeks to account for all the
    # people who haven't been confirmed yet.
    patients = patients[patients.Onset < patients.Onset.max() - pd.Timedelta(days=14)]

    return patients


def get_delays_from_patient_data(file_path=None, max_delay=60):
    patients = get_patient_data(file_path=file_path, max_delay=max_delay)
    delays = (patients.Confirmed - patients.Onset).dt.days
    delays = delays.reset_index(drop=True)
    delays = delays[delays.le(max_delay)]
    return delays


def get_delay_distribution():
    """ Returns the empirical delay distribution between symptom onset and
        confirmed positive case. """

    # The literature suggests roughly 5 days of incubation before becoming
    # having symptoms. See:
    # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7081172/
    INCUBATION_DAYS = 5

    try:
        p_delay_path = "p_delay.csv"
        p_delay = pd.read_csv(p_delay_path)
        p_delay = p_delay.iloc[:, 0]
    except FileNotFoundError:
        delays = get_delays_from_patient_data()
        p_delay = delays.value_counts().sort_index()
        new_range = np.arange(0, p_delay.index.max() + 1)
        p_delay = p_delay.reindex(new_range, fill_value=0)
        p_delay /= p_delay.sum()
        p_delay = (
            pd.Series(np.zeros(INCUBATION_DAYS))
            .append(p_delay, ignore_index=True)
            .rename("p_delay")
        )
        p_delay.to_csv("data/p_delay.csv", index=False)

    return p_delay

Возьмём код из репы и обновим до новой версии библиотеки

In [ ]:
class GenerativeModel:
    version = "1.0.0"

    def __init__(self, region: str, observed: pd.DataFrame, buffer_days=10):
        """ Takes a region (ie State) name and observed new positive and
            total test counts per day. buffer_days is the default number of
            blank days we pad on the leading edge of the time series because
            infections occur long before reports and we need to infer values
            on those days """

        first_index = observed.positive.ne(0).argmax()
        observed = observed.iloc[first_index:]
        new_index = pd.date_range(
            start=observed.index[0] - pd.Timedelta(days=buffer_days),
            end=observed.index[-1],
            freq="D",
        )
        observed = observed.reindex(new_index, fill_value=0)

        self._trace = None
        self._inference_data = None
        self.model = None
        self.observed = observed
        self.region = region

    @property
    def n_divergences(self):
        """ Returns the number of divergences from the current trace """
        assert self.trace != None, "Must run sample() first!"
        return self.trace["diverging"].nonzero()[0].size

    @property
    def inference_data(self):
        """ Returns an Arviz InferenceData object """
        assert self.trace, "Must run sample() first!"

        with self.model:
            posterior_predictive = pm.sample_posterior_predictive(self.trace)

        # _inference_data = az.from_pymc3(
        _inference_data = az.from_pymc( # ну тут понятно
            trace=self.trace,
            posterior_predictive=posterior_predictive,
        )
        _inference_data.posterior.attrs["model_version"] = self.version

        return _inference_data

    @property
    def trace(self):
        """ Returns the trace from a sample() call. """
        assert self._trace, "Must run sample() first!"
        return self._trace

    def _scale_to_positives(self, data):
        """ Scales a time series to have the same mean as the observed positives
            time series. This is useful because many of the series we infer are
            relative to their true values so we make them comparable by putting
            them on the same scale. """
        scale_factor = self.observed.positive.mean() / np.mean(data)
        return scale_factor * data

    def _get_generation_time_interval(self):
        """ Create a discrete P(Generation Interval)
            Source: https://www.ijidonline.com/article/S1201-9712(20)30119-3/pdf """
        mean_si = 4.7
        std_si = 2.9
        mu_si = np.log(mean_si ** 2 / np.sqrt(std_si ** 2 + mean_si ** 2))
        sigma_si = np.sqrt(np.log(std_si ** 2 / mean_si ** 2 + 1))
        dist = sps.lognorm(scale=np.exp(mu_si), s=sigma_si)

        # Discretize the Generation Interval up to 20 days max
        g_range = np.arange(0, 20)
        gt = pd.Series(dist.cdf(g_range), index=g_range)
        gt = gt.diff().fillna(0)
        gt /= gt.sum()
        gt = gt.values
        return gt

    def _get_convolution_ready_gt(self, len_observed):
        """ Speeds up theano.scan by pre-computing the generation time interval
            vector. Thank you to Junpeng Lao for this optimization.
            Please see the outbreak simulation math here:
            https://staff.math.su.se/hoehle/blog/2020/04/15/effectiveR0.html """
        gt = self._get_generation_time_interval()
        convolution_ready_gt = np.zeros((len_observed - 1, len_observed))
        for t in range(1, len_observed):
            begin = np.maximum(0, t - len(gt) + 1)
            slice_update = gt[1 : t - begin + 1][::-1]
            convolution_ready_gt[
                t - 1, begin : begin + len(slice_update)
            ] = slice_update
        # convolution_ready_gt = theano.shared(convolution_ready_gt)
        convolution_ready_gt = pytensor.shared(convolution_ready_gt)  # на PyTensor просто
        return convolution_ready_gt

    def build(self):
        """ Builds and returns the Generative model. Also sets self.model """

        p_delay = get_delay_distribution()
        nonzero_days = self.observed.total.gt(0)
        len_observed = len(self.observed)
        convolution_ready_gt = self._get_convolution_ready_gt(len_observed)
        x = np.arange(len_observed)[:, None]

        coords = {
            "date": self.observed.index.values,
            "nonzero_date": self.observed.index.values[self.observed.total.gt(0)],
        }
        with pm.Model(coords=coords) as self.model:

            # Let log_r_t walk randomly with a fixed prior of ~0.035. Think
            # of this number as how quickly r_t can react.
            log_r_t = pm.GaussianRandomWalk(
                "log_r_t",
                sigma=0.035,
                dims=["date"]
            )
            r_t = pm.Deterministic("r_t", pm.math.exp(log_r_t), dims=["date"])

            # For a given seed population and R_t curve, we calculate the
            # implied infection curve by simulating an outbreak. While this may
            # look daunting, it's simply a way to recreate the outbreak
            # simulation math inside the model:
            # https://staff.math.su.se/hoehle/blog/2020/04/15/effectiveR0.html
            seed = pm.Exponential("seed", 1 / 0.02)

            # вот тут главная часть начинается
            # y0 = tt.zeros(len_observed)
            # y0 = tt.set_subtensor(y0[0], seed)
            # outputs, _ = theano.scan(
            #     fn=lambda t, gt, y, r_t: tt.set_subtensor(y[t], tt.sum(r_t * y * gt)),
            #     sequences=[tt.arange(1, len_observed), convolution_ready_gt],
            #     outputs_info=y0,
            #     non_sequences=r_t,
            #     n_steps=len_observed - 1,
            # )

            # просто tt в PyTensor и scan из PyTensor
            y0 = pt.zeros(len_observed)
            y0 = pt.set_subtensor(y0[0], seed)
            outputs, _ = scan(
                fn=lambda t, gt, y, r_t: pt.set_subtensor(y[t], pt.sum(r_t * y * gt)),
                sequences=[pt.arange(1, len_observed), convolution_ready_gt],
                outputs_info=y0,
                non_sequences=r_t,
                n_steps=len_observed - 1,
            )


            infections = pm.Deterministic("infections", outputs[-1], dims=["date"])


            # Convolve infections to confirmed positive reports based on a known
            # p_delay distribution. See patients.py for details on how we calculate
            # this distribution.
            # test_adjusted_positive = pm.Deterministic(
            #     "test_adjusted_positive",
            #     conv2d(
            #         tt.reshape(infections, (1, len_observed)),
            #         tt.reshape(p_delay, (1, len(p_delay))),
            #         border_mode="full",
            #     )[0, :len_observed],
            #     dims=["date"]
            # )



            # Из help выше понятно, что conv2d требует 4D (batch, in_ch, rows, cols)
            test_adjusted_positive = pm.Deterministic(
                "test_adjusted_positive",
                conv2d(
                    pt.reshape(infections, (1, 1, 1, len_observed)),   # вот как раз 4D
                    pt.reshape(p_delay,   (1, 1, 1, len(p_delay))),     # и тут 4D
                    border_mode="full",
                )[0, 0, 0, :len_observed],                              # а теперь берём 1D свёртку по времени
                dims=["date"]
            )


            # Picking an exposure with a prior that exposure never goes below
            # 0.1 * max_tests. The 0.1 only affects early values of Rt when
            # testing was minimal or when data errors cause underreporting
            # of tests.
            tests = pm.Data("tests", np.ones(len_observed, dtype=float), dims=["date"])
            exposure = pm.Deterministic("exposure", tests, dims=["date"])

            # Test-volume adjust reported cases based on an assumed exposure
            # Note: this is similar to the exposure parameter in a Poisson
            # regression.
            positive = pm.Deterministic("positive", exposure * test_adjusted_positive, dims=["date"])

            # Save data as part of trace so we can access in inference_data
            observed_positive = pm.Data("observed_positive", self.observed.positive.values, dims=["date"])
            nonzero_observed_positive = pm.Data("nonzero_observed_positive", self.observed.positive[nonzero_days.values].values, dims=["nonzero_date"])

            positive_nonzero = pm.NegativeBinomial(
                "nonzero_positive",
                mu=positive[nonzero_days.values],
                alpha=pm.Gamma("alpha", mu=6, sigma=1),
                observed=nonzero_observed_positive,
                dims=["nonzero_date"]
            )

        return self.model

    def sample(
        self,
        cores=4,
        chains=4,
        tune=700,
        draws=200,
        target_accept=0.95,
        init="jitter+adapt_diag",
    ):
        """ Runs the PyMC3 model and stores the trace result in self.trace """
        # ну тут понятно, что PyMC5 model

        if self.model is None:
            self.build()

        with self.model:
            self._trace = pm.sample(
                draws=draws,
                cores=cores,
                chains=chains,
                target_accept=target_accept,
                tune=tune,
                init=init,
            )

        return self

In [ ]:
import plotly.graph_objects as go

In [ ]:
TRAIN_START = pd.Timestamp("2020-01-01")
TRAIN_END   = pd.Timestamp("2020-12-01")
TEST_START  = pd.Timestamp("2020-12-02")
TEST_END    = pd.Timestamp("2020-12-14")

In [ ]:
region = "Russia"
d_train = (df_res[(df_res["location"]==region) &
                  (df_res["date"]>=TRAIN_START) &
                  (df_res["date"]<=TRAIN_END)][["date","new_cases"]]
           .rename(columns={"new_cases":"positive"}))
d_train["total"] = 10.0
d_train = (d_train.set_index("date").asfreq("D").fillna(0.0)
                  .astype({"positive":float,"total":float}))

# будущее
d_test = (df_res[(df_res["location"]==region) &
                 (df_res["date"]>=TEST_START) &
                 (df_res["date"]<=TEST_END)][["date","new_cases"]]
          .set_index("date").asfreq("D").fillna(0.0))
y_test = d_test["new_cases"].astype(float)

future_idx = pd.date_range(d_train.index[-1] + pd.Timedelta(days=1), TEST_END, freq="D")
observed_ext = pd.concat([
    d_train,
    pd.DataFrame({"positive": 0.0, "total": 0.0}, index=future_idx)  # total ставлю 0, чтобы дни теста не в правдоподобии были
], axis=0)

In [ ]:
gm = GenerativeModel(region, observed_ext)
gm.sample(cores=4, chains=4, tune=300, draws=200, init="jitter+adapt_diag")

In [ ]:
da = gm.trace.posterior["positive"].mean(dim=("chain","draw"))
pred_series = pd.Series(np.asarray(da), index=pd.to_datetime(np.asarray(da["date"])))

actual_train = observed_ext.loc[observed_ext["total"] > 0, "positive"]
fig = go.Figure()
fig.add_trace(go.Scatter(x=actual_train.index, y=actual_train.values, name="Факт (train)"))
fig.add_trace(go.Scatter(x=y_test.index,      y=y_test.values,       name="Факт (test)"))
fig.add_trace(go.Scatter(x=pred_series.index, y=pred_series.values,  name="Оценка"))
fig.update_layout(title=f"{region}: оценка vs факт",
                  xaxis_title="Дата", yaxis_title="Число зарегистрированных случаев в день")
fig.show()

In [ ]:
region = "France"
d_train = (df_res[(df_res["location"]==region) &
                  (df_res["date"]>=TRAIN_START) &
                  (df_res["date"]<=TRAIN_END)][["date","new_cases"]]
           .rename(columns={"new_cases":"positive"}))
d_train["total"] = 10.0
d_train = (d_train.set_index("date").asfreq("D").fillna(0.0)
                  .astype({"positive":float,"total":float}))

# будущее
d_test = (df_res[(df_res["location"]==region) &
                 (df_res["date"]>=TEST_START) &
                 (df_res["date"]<=TEST_END)][["date","new_cases"]]
          .set_index("date").asfreq("D").fillna(0.0))
y_test = d_test["new_cases"].astype(float)

future_idx = pd.date_range(d_train.index[-1] + pd.Timedelta(days=1), TEST_END, freq="D")
observed_ext = pd.concat([
    d_train,
    pd.DataFrame({"positive": 0.0, "total": 0.0}, index=future_idx)  # total ставлю 0, чтобы дни теста не в правдоподобии были
], axis=0)

In [ ]:
gm = GenerativeModel(region, observed_ext)
gm.sample(cores=4, chains=4, tune=300, draws=200, init="jitter+adapt_diag")

/usr/local/lib/python3.12/dist-packages/pymc/distributions/timeseries.py:297: UserWarning:

Initial distribution not specified, defaulting to `Normal.dist(0, 100)`.You can specify an init_dist manually to suppress this warning.



Output()

In [ ]:
da = gm.trace.posterior["positive"].mean(dim=("chain","draw"))
pred_series = pd.Series(np.asarray(da), index=pd.to_datetime(np.asarray(da["date"])))

actual_train = observed_ext.loc[observed_ext["total"] > 0, "positive"]
fig = go.Figure()
fig.add_trace(go.Scatter(x=actual_train.index, y=actual_train.values, name="Факт (train)"))
fig.add_trace(go.Scatter(x=y_test.index,      y=y_test.values,       name="Факт (test)"))
fig.add_trace(go.Scatter(x=pred_series.index, y=pred_series.values,  name="Оценка"))
fig.update_layout(title=f"{region}: оценка vs факт",
                  xaxis_title="Дата", yaxis_title="Число зарегистрированных случаев в день")
fig.show()

In [ ]:
region = "Germany"
d_train = (df_res[(df_res["location"]==region) &
                  (df_res["date"]>=TRAIN_START) &
                  (df_res["date"]<=TRAIN_END)][["date","new_cases"]]
           .rename(columns={"new_cases":"positive"}))
d_train["total"] = 10.0
d_train = (d_train.set_index("date").asfreq("D").fillna(0.0)
                  .astype({"positive":float,"total":float}))

# будущее
d_test = (df_res[(df_res["location"]==region) &
                 (df_res["date"]>=TEST_START) &
                 (df_res["date"]<=TEST_END)][["date","new_cases"]]
          .set_index("date").asfreq("D").fillna(0.0))
y_test = d_test["new_cases"].astype(float)

future_idx = pd.date_range(d_train.index[-1] + pd.Timedelta(days=1), TEST_END, freq="D")
observed_ext = pd.concat([
    d_train,
    pd.DataFrame({"positive": 0.0, "total": 0.0}, index=future_idx)  # total ставлю 0, чтобы дни теста не в правдоподобии были
], axis=0)

In [ ]:
gm = GenerativeModel(region, observed_ext)
gm.sample(cores=4, chains=4, tune=300, draws=200, init="jitter+adapt_diag")

/usr/local/lib/python3.12/dist-packages/pymc/distributions/timeseries.py:297: UserWarning:

Initial distribution not specified, defaulting to `Normal.dist(0, 100)`.You can specify an init_dist manually to suppress this warning.



Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


In [ ]:
da = gm.trace.posterior["positive"].mean(dim=("chain","draw"))
pred_series = pd.Series(np.asarray(da), index=pd.to_datetime(np.asarray(da["date"])))

actual_train = observed_ext.loc[observed_ext["total"] > 0, "positive"]
fig = go.Figure()
fig.add_trace(go.Scatter(x=actual_train.index, y=actual_train.values, name="Факт (train)"))
fig.add_trace(go.Scatter(x=y_test.index,      y=y_test.values,       name="Факт (test)"))
fig.add_trace(go.Scatter(x=pred_series.index, y=pred_series.values,  name="Оценка"))
fig.update_layout(title=f"{region}: оценка vs факт",
                  xaxis_title="Дата", yaxis_title="Число зарегистрированных случаев в день")
fig.show()

In [ ]:
region = "Italy"
d_train = (df_res[(df_res["location"]==region) &
                  (df_res["date"]>=TRAIN_START) &
                  (df_res["date"]<=TRAIN_END)][["date","new_cases"]]
           .rename(columns={"new_cases":"positive"}))
d_train["total"] = 10.0
d_train = (d_train.set_index("date").asfreq("D").fillna(0.0)
                  .astype({"positive":float,"total":float}))

# будущее
d_test = (df_res[(df_res["location"]==region) &
                 (df_res["date"]>=TEST_START) &
                 (df_res["date"]<=TEST_END)][["date","new_cases"]]
          .set_index("date").asfreq("D").fillna(0.0))
y_test = d_test["new_cases"].astype(float)

future_idx = pd.date_range(d_train.index[-1] + pd.Timedelta(days=1), TEST_END, freq="D")
observed_ext = pd.concat([
    d_train,
    pd.DataFrame({"positive": 0.0, "total": 0.0}, index=future_idx)  # total ставлю 0, чтобы дни теста не в правдоподобии были
], axis=0)

In [ ]:
gm = GenerativeModel(region, observed_ext)
gm.sample(cores=4, chains=4, tune=300, draws=200, init="jitter+adapt_diag")

/usr/local/lib/python3.12/dist-packages/pymc/distributions/timeseries.py:297: UserWarning:

Initial distribution not specified, defaulting to `Normal.dist(0, 100)`.You can specify an init_dist manually to suppress this warning.



Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


In [ ]:
da = gm.trace.posterior["positive"].mean(dim=("chain","draw"))
pred_series = pd.Series(np.asarray(da), index=pd.to_datetime(np.asarray(da["date"])))

actual_train = observed_ext.loc[observed_ext["total"] > 0, "positive"]
fig = go.Figure()
fig.add_trace(go.Scatter(x=actual_train.index, y=actual_train.values, name="Факт (train)"))
fig.add_trace(go.Scatter(x=y_test.index,      y=y_test.values,       name="Факт (test)"))
fig.add_trace(go.Scatter(x=pred_series.index, y=pred_series.values,  name="Оценка"))
fig.update_layout(title=f"{region}: оценка vs факт",
                  xaxis_title="Дата", yaxis_title="Число зарегистрированных случаев в день")
fig.show()